In [1]:
!pip install kafka-python



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 54.9 MB/s eta 0:00:00


In [3]:

from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
import mysql.connector
from mysql.connector import Error
from datetime import datetime

In [5]:
consumer = KafkaConsumer(
    'demo',
    bootstrap_servers=['20.204.133.198:9092'], #add your IP here
    auto_offset_reset='latest',  # Start from the earliest message
    enable_auto_commit=True,
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [6]:
try:
  conn = mysql.connector.connect(
      host='20.204.133.198',    # Remote MySQL server address
      user='stock',           # MySQL username
      password='stock',       # MySQL password
      database='stockdb',       # The database you want to connect to
      port=3306               # Default MySQL port (you can change it if necessary)
  )
except Error as e:
  print(f"Error connecting to MySQL: {e}")


In [7]:

while True:
  sleep(10)
  msg = consumer.poll(timeout_ms=10000)
  if msg:
    for topic_partition, messages_list in msg.items():
      print("\n\nBatch count: ", len(messages_list), "\n\n")

    cursor = conn.cursor()
    # Insert statement
    insert_query = """
    INSERT INTO StockData (IndexName, Date, Open, High, Low, Close, AdjClose, Volume, CloseUSD)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    for message in messages_list:
      print(f"Consumed: {message.value}")

      json_object = message.value

      json_object['Date'] = datetime.strptime(json_object['Date'], '%Y-%b-%d').strftime('%Y-%m-%d')

      cursor.execute(insert_query, (
          json_object['Index'],
          json_object['Date'],
          json_object['Open'],
          json_object['High'],
          json_object['Low'],
          json_object['Close'],
          json_object['Adj Close'],  # Use underscore for the column name
          json_object['Volume'],
          json_object['CloseUSD']
      ))

    conn.commit()
    cursor.close()
  else:
    print("No message from producer")





Batch count:  1 


Consumed: {'Index': 'N225', 'Date': '1973-JAN-09', 'Open': 5278.100098, 'High': 5278.100098, 'Low': 5278.100098, 'Close': 5278.100098, 'Adj Close': 5278.100098, 'Volume': 0.0, 'CloseUSD': 52.78100098}


ProgrammingError: 1146 (42S02): Table 'stockdb.StockData' doesn't exist

In [ ]:
conn.close()